![Assimila logo](../../helpers/logosmall.png "Assimila")
# Compare bounding box for 2 time steps

In [2]:
import sys
sys.path.append("../../helpers")

from data import Data
from maptools import MapTools
from widgets import Widgets

import matplotlib
%matplotlib inline
matplotlib.use("nbagg")
from IPython.display import clear_output
from IPython.core.display import HTML
from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles, 
                        TileLayer, ImageOverlay, Polyline, Polygon, 
                        Rectangle,  GeoJSON, WidgetControl,  DrawControl, 
                        LayerGroup, FullScreenControl, interactive)

import ipywidgets as widgets
import datetime

display(HTML("""
<style>
.container { 
    width:60% !important; 
}
</style>
"""))

display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))


def on_button_clicked(b):
    button.disabled = True
    d = Data(out)
    m1 = d.color_map_nesw_compare(product.value, 
                                  subproduct.value, 
                                  north.value,
                                  east.value, 
                                  south.value, 
                                  west.value,
                                  date1, 
                                  hour1, 
                                  date2, 
                                  hour2)
    button.disabled = False

def update_north(x):
    """"
    Updates the 'North' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (north, 'value'))

def update_east(x):
    """"
    Updates the 'East' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (east, 'value'))

def update_south(x):
    """"
    Updates the 'South' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (south, 'value'))

def update_west(x):
    """"
    Updates the 'West' coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (west, 'value'))    
            
        
def handle_draw(self, action, geo_json):
    """
    Handles drawing of layers on the map by extracting coordinates
    and updating the relevant coordinate display box.
    
    :param action: 
    :param geo_json: 
    
    :return:
    """
    w = Widgets()
    d = Data(out)
    
    s = geo_json.get('geometry','type')
    t = s.get('type')
    
    if t == 'Point':
        north, east, south, west = MapTools.get_coords_point(geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

    if t == 'Polygon':
        north, east, south, west = MapTools.get_coords_polygon(geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

def update(*args):
    subproduct.options = w.get_subproduct_list(product.value)

m = MapTools(center=(1, -5), zoom=2, width='380px', height='500px')
m.map.add_control(m.dc)
m.prepare_map()

w = Widgets()
product = w.product()
subproduct = w.subproduct()
north = w.get_point(38.0, 'North: ')
east = w.get_point(53.0, 'East: ')
south = w.get_point((-36), 'South: ')
west = w.get_point((-19), 'West: ')
date1 = w.get_date(datetime.datetime(2018,1,1), 'Date 1: ')
hour1 = w.get_hour('00', 'Hour 1: ')
date2 = w.get_date(datetime.datetime(2018,1,2), 'Date 2: ')
hour2 = w.get_hour('00', 'Hour 2: ')

date1.layout = widgets.Layout(width='auto', height='auto')
date2.layout = widgets.Layout(width='auto', height='auto')
north.layout = widgets.Layout(width='auto', height='auto')
east.layout = widgets.Layout(width='auto', height='auto')
south.layout = widgets.Layout(width='auto', height='auto')
west.layout = widgets.Layout(width='auto', height='auto')

button = w.set_up_button(on_button_clicked, "Get data")
w.display_widget_comparison(product, subproduct, north, east, south, west, 
                            date1, hour1, date2, hour2, button, m.map)

label = widgets.Label()
m.mouse_interaction(label)

update()
product.observe(update)

out = w.display_output()

m.dc.on_draw(handle_draw)

Label(value='')

Output()